In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [13]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'

In [3]:
bank_customer_data = spark.read.json("file:///home/hadoop/Downloads/bank_edited.json",multiLine=True)

In [4]:
bank_customer_data.show()

+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+
|age|balance|campaign|contact|day|default|duration|education|housing|         job|loan| marital|month|pdays|poutcome|previous|  y|
+---+-------+--------+-------+---+-------+--------+---------+-------+------------+----+--------+-----+-----+--------+--------+---+
| 58|   2143|       1|unknown|  5|     no|     261| tertiary|    yes|  management|  no| married|  may|   -1| unknown|       0| no|
| 44|     29|       1|unknown|  5|     no|     151|secondary|    yes|  technician|  no|  single|  may|   -1| unknown|       0| no|
| 33|      2|       1|unknown|  5|     no|      76|secondary|    yes|entrepreneur| yes| married|  may|   -1| unknown|       0| no|
| 47|   1506|       1|unknown|  5|     no|      92|  unknown|    yes| blue-collar|  no| married|  may|   -1| unknown|       0| no|
| 33|      1|       1|unknown|  5|     no|     198|  unknown|     no|     unknown| 

In [5]:
bank_customer_data.printSchema()

root
 |-- age: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- default: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- education: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- job: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- month: string (nullable = true)
 |-- pdays: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- previous: long (nullable = true)
 |-- y: string (nullable = true)



### 1. Display max, min and mean of age of targeted customer

In [6]:
bank_customer_data.createOrReplaceTempView("banktable")

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [8]:
bank_customer_data.select(max("age")).show()

+--------+
|max(age)|
+--------+
|      95|
+--------+



In [9]:
bank_customer_data.select(min("age")).show()

+--------+
|min(age)|
+--------+
|      18|
+--------+



In [10]:
bank_customer_data.select(mean("age")).show()

+-----------------+
|         avg(age)|
+-----------------+
|40.93621021432837|
+-----------------+



In [11]:
# sparksql - alternative method
spark.sql("select max(age),min(age),mean(age) from banktable").show()

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

### 2. Check the quality of customer by analyzing targeted customers min,max median balance

In [ ]:
spark.sql("select max(balance) as maximum,min(abs(balance)) as minimum,mean(balance) as average,percentile_approx(balance,0.5) as median from banktable").show()

### 3. Check if age matters in marketting subscription for term deposit scheme

In [ ]:
spark.sql("select age, count(*) as customer_count from banktable where y='yes' \
        group by age order by customer_count desc").show()

### 4. Calculate Marketing Success

In [ ]:
spark.sql("select round((select count(*) from banktable where y='yes')*100,2)/(select count(*) from banktable) as success_rate").show()

In [ ]:
spark.sql("select round((select count(*) from banktable where y='no')*100,2)/(select count(*) from banktable) as failure_rate").show()

### 6. Check if martial status matters in marketing subscription for term deposit scheme

In [ ]:
spark.sql("select marital,count(*) as status from banktable where y='yes' group by marital").show()

### 8. if age and martial status together mattered for subscription to term deposit scheme

In [ ]:
spark.sql("select age,marital, count(*) as customer_count from banktable where y='yes' \
        group by age,marital order by customer_count desc").show()

### 9. Compute success rate if each age and martial status category

In [ ]:
spark.sql(""" 
    select age, marital, count(*) as customer_count,
    sum(case when y='yes' then 1 else 0 end) as count_success,
    (sum(case when y='yes' then 1 else 0 end)*100/count(*)) as success_percentage
    from banktable group by age,marital
    
""").show()

### 10. Do feature engineering for bank investment scheme and find effect of age on the campaign

In [ ]:
spark.sql("""
    select age_category, count(*) as success_count from (
    select case when age<25 then 'Teenager'
                when age>=25 and age<=33 then 'Adult'
                when age>=35 and age<55 then 'Middle Age'
            else 'Old'
            end as age_category
            from banktable where y='yes' 
    ) group by age_category order by success_count desc
    
    """).show()

### alternative : creating UDF

In [ ]:
from pyspark.sql.functions import *

In [ ]:
age_range = udf(lambda age : 'Teenager' if age<20 else
                             'Adult' if (age>=25 and age<35) else
                             'Middle Aged' if (age>=35 and age<55) else 'Old')

In [ ]:
bank_df = bank_customer_data.withColumn('Age Category',age_range(bank_customer_data.age))

In [ ]:
bank_df.show()

### 11.write a query to show distributed probability rate for each category